This is a cover letter builder using agentic ai


In [39]:
#imports

from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr
from pydantic import BaseModel
import os
import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
import json

In [40]:
load_dotenv(override=True)


True

In [41]:
openai = OpenAI()

In [42]:
creator_model = "gpt-4o"
evaluator_model = "o4-mini"

In [43]:
with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()
    

In [44]:
reader = PdfReader("me/resume.pdf")
resume = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        resume += text

In [45]:
name = "Sviatoslav Rutkovskyi"


In [57]:
system_prompt = f"""You are a proffesional cover letter writer, and your job is to write a cover letter for {name}, highlighting {name}'s skills, experience, and achievements. 
particularly questions related to {name}'s career, background, skills and experience. 
Your responsibility is to represent {name} in the letter as faithfully as possible. 
You are given a summary of {name}'s background and Resume which you can use in the cover letter. 
You are given an example of a cover letter from {name} which you can use as a starting point. Try and use a similar language and style.
Be professional and engaging, uing the tone and style suitable for a cover letter.
Do not make up any information, and only use the information provided.
Don't be too verbose, and use a 3 paragraph format.
Respond with a cover letter and nothing else.
Do not include the address or contact information. 
You will be given a job description, and you will need to tailor the cover letter to the job description.
You will be evaluated, and if evalutor decides that your cover letter is not up to standart, you will be given your previus cover letters and feedback on them. 
\n\n## Summary:\n{summary}\n\n## Resume:\n{resume}\n\n
"""



In [47]:

updated_system_prompt = system_prompt

In [48]:
evaluator_system_prompt = f"""
You are a professional evaluator that decides whether a cover letter is acceptable. 
You are provided with {name}'s summary and resume, the job description, and the cover letter. 
Your task is to evaluate the cover letter, and reply with whether it is acceptable and your feedback. 
You need to ensure if the cover letter is professional, engaging, and tailored to the job description. 
You need to ensure if the cover letter was likely made by AI, and if it was made by AI, deny it, and provide feedback. Do not allow AI generated cover letters.
You need to ensure that the cover letter has a strong and engaging opening paragraph. 
You need to ensure that the cover letter does not include any markdown and formatting besides the stardart 3 paragraph format. 
You need to ensure that the cover letter is not too verbose, and is concise.
Here's the information:
\n\n## Summary:\n{summary}\n\n## Resume:\n{resume}\n\n
With this context, please evaluate the cover letter, replying with whether the cover letter is acceptable and your feedback.
"""

In [49]:
class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str

In [50]:
def evaluator_cover_letter(job_post, cover_letter):
    return f"""
    Here's the job posting presented by the user: \n\n{job_post}\n\n
    Here's the cover letter generated by the agent: \n\n{cover_letter}\n\n
    Please evaluate the response, replying with whether it is acceptable and your extensive feedback.
    """

In [51]:
def update_system_prompt(cover_letter, feedback):
        global updated_system_prompt
        updated_system_prompt = updated_system_prompt + f"""
        \n\n## Previous cover letter rejected\nYou just tried to create a cover letter, but the quality control rejected your cover letter\n
        ## Your attempted cover letter:\n{cover_letter}\n\n
        ## Reason for rejection:\n{feedback}\n\n
        """
        return updated_system_prompt;

In [52]:
def evaluate(job_post, cover_letter) -> Evaluation:
    messages = [
        {"role": "system", "content": evaluator_system_prompt},
        {"role": "user", "content": evaluator_cover_letter(job_post, cover_letter)},
        {"role": "user", "content": "Reply ONLY in valid JSON: {\"is_acceptable\": true/false, \"feedback\": \"...\"}"}
    ]
    response = openai.responses.parse(
        model=evaluator_model,
        reasoning = {"effort":"high"},
        input=messages,
        text_format = Evaluation,
        )
    return response.output_parsed

In [53]:
def run(prompt, job_posting):
    messages = [{"role": "system", "content": prompt}] + [{"role": "user", "content": job_posting}]
    response = openai.chat.completions.create(model=creator_model, messages=messages)
    return response.choices[0].message.content

In [54]:

def scrape_webpage_simple(url):
    try:
        # Set headers to mimic a real browser
        headers = {
            'User-Agent': 'Mozilla/5.0'
        }
        
        # Make the request
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()  # Raise an error for bad status codes
        
        # Parse the HTML
        soup = BeautifulSoup(response.content, 'html.parser')

        # Get all text content
        return soup.get_text()
        
        
    except requests.RequestException:
        return 'error'
    except Exception:
        return 'error'


In [55]:
def requestLetter(job_posting, history):
    global updated_system_prompt
    page = scrape_webpage_simple(job_posting)
    print(page)
    if page == 'error':
        print("Failed to scrape job posting")
    else:
        job_posting = page

    cover_letter = run(system_prompt, job_posting)

    eval_counter = 0
    while eval_counter < 10:
        evaluation = evaluate(job_posting, cover_letter)
        if evaluation.is_acceptable:
            print("Passed evaluation - returning reply")
            display(Markdown(f"## Cover Letter:\n{cover_letter}"));
            display(Markdown(f"## Feedback:\n{evaluation.feedback}"))
            display(Markdown(f"## Updated system prompt:\n{updated_system_prompt}"))
            updated_system_prompt = system_prompt;
            return cover_letter
        else:
            eval_counter += 1
            print("Failed evaluation - retrying")
            display(Markdown(f"## Cover Letter:\n{cover_letter}"))
            display(Markdown(f"## Feedback:\n{evaluation.feedback}"))
            cover_letter = run(update_system_prompt(cover_letter, evaluation.feedback), job_posting)
    print("Failed evaluation - returning reply")
    return "Unable to generate cover letter" +"\n" + evaluation.feedback    
    

In [56]:
gr.ChatInterface(requestLetter, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


error
Failed to scrape job posting
Passed evaluation - returning reply


## Cover Letter:
I am writing to express my interest in the Software Engineer - Summer Internship role at Bits. With a Bachelor of Science in Computer Science from the University of Washington Tacoma, and extensive hands-on experience in software development projects, I am eager to contribute to your mission of revolutionizing credit building while gaining invaluable insights into the fintech space. My strong academic background, complemented by a cumulative GPA of 3.9, reflects my quick learning ability and dedication to excelling in challenging environments.

During my academic career, I have acquired a solid foundation in programming languages and frameworks, including Java, JavaScript, Python, and TypeScript, as well as experience in web development tools and version control systems such as Git. My project experience ranges from implementing advanced cryptographic algorithms in a Java application to developing full-stack web applications with Node.js and React.js. Furthermore, as a project leader in a Machine Learning course, I honed my problem-solving and teamwork skills, achieving significant predictive accuracy improvements for user demographics analysis. These experiences have equipped me with the technical acumen and collaborative spirit to effectively contribute to the engineering team at Bits.

I am particularly drawn to this opportunity at Bits because of your commitment to creating meaningful, high-impact solutions for underserved communities. My passion for innovative technology and dedication to making a positive change align seamlessly with your mission. As an intern, I am eager to leverage my technical skills, proactive attitude, and adaptability in contributing to real projects that impact your users daily. I look forward to the opportunity to work closely with your team and founder, embracing new challenges within the dynamic environment at Bits. Thank you for considering my application.

## Feedback:
This cover letter is well-structured, professional, and tailored to Bits’ mission. It follows the required three-paragraph format, makes no use of markdown, and highlights relevant academic achievements, technical skills (Java, JavaScript, Python, TypeScript, Git), and project leadership—qualities that match the internship requirements. The third paragraph clearly connects your passion to the company’s goal of serving underserved communities, demonstrating genuine interest.

To further strengthen it, consider opening with a more engaging hook that ties your personal motivations directly to Bits’ mission (for example, a brief anecdote or observation about credit accessibility). You could also incorporate the “Desired Extras” by mentioning any hands-on experience you have with Node.js/TypeScript backends, AWS or other cloud platforms, and SQL databases. Finally, a brief sentence emphasizing your ability to thrive in a remote, part-time internship setting would reinforce your fit for this role.

## Updated system prompt:
You are a proffesional cover letter writer, and your job is to write a cover letter for Sviatoslav Rutkovskyi, highlighting Sviatoslav Rutkovskyi's skills, experience, and achievements. 
particularly questions related to Sviatoslav Rutkovskyi's career, background, skills and experience. 
Your responsibility is to represent Sviatoslav Rutkovskyi in the letter as faithfully as possible. 
You are given a summary of Sviatoslav Rutkovskyi's background and Resume which you can use in the cover lettter. 
Be professional and engaging, using the tone and style suitable for a cover letter.
Do not make up any information, and only use the information provided.
Don't be too verbose, and use a 3 paragraph format.
Respond with a cover letter and nothing else.
Do not include the address or contact information. 
You will be given a job description, and you will need to tailor the cover letter to the job description.
You will be evaluated, and if evalutor decides that your cover letter is not up to standart, you will be given your previus cover letters and feedback on them. 


## Summary:
My name is Sviatoslav Rutkovskyi, and I am a recent graduate from university of Washington Tacoma, with a Bachelor of Science in Computer Science, and 3.9 cumulative GPA. I also have a minor in Mathematics. I am an aspiring software developer, and want to use my skills to make a positive change in the world. 
My hobbies include playing guitar, hiking, and skiing. I bought skiing gear in April, and I'm excited to give them a test drive once the season arrives.

My interest in coding started in highschool, when I would write simple python code in my free time. During my running start at Bellevue college, I decided to keep pursing my inteterest in programming, and took some introductory computer science classes, and worked on getting my prerequisites completed for a Bachelor of Science in Computer Science. After finishing running start, I quickly joined the army reserves, and completed the bootcamp. 
After returning from the army bootcamp, I started preparing for the next step in my education, while taking online courses on web development and PHP on the side. I decided to go to the University of Washington, Tacoma campus to continue my higher education. At the same time, I was still working part time at home depot, while maintaining high grades. During my Senior year, I quit my job at Home Depot to focus more on my education, even though my supervisors wanted me to stay. I was an employee who always did the work that was required of me, and went above and beyond to do so. 
I took many courses while in university, incuding: Data Structures, Web Development, Cryptography, Programming Practicum, Discrete Structures, Software Development and Quality Assurance Techniques, Computer Operating Systems, Artificial Intelligence and Knowledge Acquisition, Database Systems Design, 
Machine Learning, and Algorithms. I have gained many skills from these classes, and completed many projects, which are summarized below. 

Projects: 

Asymmetric Encryption App
Completion Date: December 2024
Course: Cryptography
• Implemented advanced cryptographic algorithms including SHA3 (sponge and SHAKE functions) and an
elliptic-curve encryption scheme (ECIES) in a Java application.
• Utilizes SHA3 functions to compute hashes, authentication tags, and encrypt a user-specified file, as well as
decrypt the cryptogram and verify the MAC
• Utilizes ECIES functions to create public keys, encrypt a file under the public key sign it with Schnorr, as well
as to decrypt the file with a private key, and verify it.
• Tech Stack: Java, IntelliJ, and Github.


Bookkeeping Web App 
Completion Date: July 2024
Course: Web Development
• Full-stack web application for managing a book catalog (view/add/remove books and update ratings) with
secure user registration, login, and role-based access control.
• Collaborated in a team of 5 to design and document a RESTful API backend (Node.js, Express, PostgreSQL),
using Docker for containerization and tools like ApiDoc/Postman for API documentation and testing.
• Adapted to a new codebase by swapping projects with another team and developing a React/Next.js front-end
that consumed their API, demonstrating flexibility and teamwork to deliver the project successfully.
• My Role: Team Leader, created and managed backend repository, resolved merges and pull requests,
implemented POST for backend and frontend, communicated with other teams, fixed various bugs.
• Tech Stack: Typescript, React.js, Next.js, Docker, ApiDoc, Postgres, Postman, WebStorm.


Social Media Data Neural Network
Completion Date: July 2024
Course: Machine Learning
• Built and evaluated machine learning models (Naive Bayes, linear regression, and deep neural networks) to predict user demographics (age group and gender) from social media data.
• Achieved prediction accuracies of 64% (age group) and 82% (gender) using a neural network, improving significantly over the baseline accuracy (˜59%) and showcasing strong analytical and problem-solving skills.
• My Role: Project Leader, created baseline results, worked on the likes dataset, assisted other team members.
• Tech Stack: Python, scikit-learn, Keras, TensorFlow, Ubuntu VM, PyCharm.


Dungeon Crawler
Completion Date: December 2023
Course: Software Development and Quality Assurance Techniques
• Developed a Java Swing dungeon crawler game featuring randomly generated dungeons with monsters and collectible items, plus multiple playable heroes.
• Implemented game mechanics including a minimap, inventory system, turn-based combat, and save/load functionality, following an MVC architecture and other design patterns (Singleton, Factory) for clean, maintainable code.
• Created thorough JUnit testing for the entire back-end model.
• Created a UML diagram for the project, used Pivotal Tracker and Toggl to organize and delegate tasks, and to track the time spent on each task.
• My Role: Team Leader, created and maintained the repository, created most of the model, including recursive dungeon generation and the inventory system, and worked on the View-Controller.
• Tech Stack: Java, Swing, IntelliJ, GitHub, Pivotal Tracker, Toggl



 

## Resume:
Sviatoslav Rutkovskyi
Kent, Washington
/ne571-992-8193 rutkovsky2003@gmail.com LinkedIn: sviatoslav-rutkovskyi
PROFILE
Highly motivated software engineer looking to work my dream job, equipped with quick learning ability proven by
academic success. Able to work under pressure, provide outstanding leadership, and maintain personal and
professional integrity, given my experience as a sergeant in the US Army Reserves. Passionate about new
technology within software engineering such as AI and ML. I enjoy hiking and playing guitar as hobbies.
TECHNICAL SKILLS
Programming Languages:Python, Java, C, C#, JavaScript, SQL, R, HTML/CSS
Developer Tools:VS Code, MySQL Workbench, R Studio, IntelliJ, PyCharm, WebStorm
Technologies/Frameworks:GitHub, ReactJS, NodeJS, ExpressJS, Git, Docker, Postman
Soft Skills:Quick Learning Ability, Communication, Logical Thinking, Task Management, Problem-Solving
Spoken Languages:Native Speaker of Ukrainian, Fluent in English and Russian
Relevant Coursework:ML, AI, Data Structures & Algorithms, Databases, Client/Server, Cryptography
PROJECTS
Asymmetric Encryption App| Github 12 2024
• Implemented advanced cryptographic algorithms including SHA3 (sponge and SHAKE functions) and an
elliptic-curve encryption scheme (ECIES) in a Java application.
• Utilizes SHA3 functions to compute hashes, authentication tags, and encrypt a user-specified file, as well as
decrypt the cryptogram and verify the MAC
• Utilizes ECIES functions to create public keys, encrypt a file under the public key sign it with Schnorr, as well
as to decrypt the file with a private key, and verify it.
• Tech Stack: Java, IntelliJ, and Github.
Bookkeeping Web App| Github Backend Github Frontend 06 2024
• Full-stack web application for managing a book catalog (view/add/remove books and update ratings) with
secure user registration, login, and role-based access control.
• Collaborated in a team of 5 to design and document a RESTful API backend (Node.js, Express, PostgreSQL),
using Docker for containerization and tools like ApiDoc/Postman for API documentation and testing.
• Adapted to a new codebase by swapping projects with another team and developing a React/Next.js front-end
that consumed their API, demonstrating flexibility and teamwork to deliver the project successfully.
• My Role: Team Leader, created and managed backend repository, resolved merges and pull requests,
implemented POST for backend and frontend, communicated with other teams, fixed various bugs.
• Tech Stack: Typescript, React.js, Next.js, Docker, ApiDoc, Postgres, Postman, WebStorm.
Social Media Data Neural Network 06 2024
• Built and evaluated machine learning models (Naive Bayes, linear regression, and deep neural networks) to
predict user demographics (age group and gender) from social media data.
• Achieved prediction accuracies of 64% (age group) and 82% (gender) using a neural network, improving
significantly over the baseline accuracy ( ˜59%) and showcasing strong analytical and problem-solving skills.
• My Role: Project Leader, created baseline results, worked on the likes dataset, assisted other team members.
• Tech Stack: Python, scikit-learn, Keras, TensorFlow, Ubuntu VM, PyCharm.
Dungeon Crawler 12 2023
• Developed a Java Swing dungeon crawler game featuring randomly generated dungeons with monsters and
collectible items, plus multiple playable heroes.
• Implemented game mechanics including a minimap, inventory system, turn-based combat, and save/load
functionality, following an MVC architecture and other design patterns (Singleton, Factory) for clean,
maintainable code.
• Created thorough JUnit testing for the entire back-end model.
• Created a UML diagram for the project, used Pivotal Tracker and Toggl to organize and delegate tasks, and to
track the time spent on each task.
• My Role: Team Leader, created and maintained the repository, created most of the model, including recursive
dungeon generation and the inventory system, and worked on the View-Controller.
• Tech Stack: Java, Swing, IntelliJ, GitHub, Pivotal Tracker, TogglEXPERIENCE
US Army Reserve 07 2021 – Present
Sergeant, Bridge Crew Member, 12C Marysville, Washington
• Demonstrated integrity, dedication, and mental fortitude through military training and duties, allowing me to
perform under pressure.
• Taught classes up to and exceeding army standards and provided mentorship to lower enlisted soldiers, setting
them up for success.
• Led and collaborated with team members in high-pressure environments, honing strong leadership and
teamwork skills that translate to managing projects and coordinating with colleagues in tech settings.
Home Depot 07 2022 – 01 2024
Sales Associate Kent, Washington
• Quickly learned new skills and honed my communication skills, rapidly increasing my knowledge in new areas
• Provided excellent customer service to a diverse customer base, in a timely and efficient manner.
• Maintained a positive team environment while consistently meeting sales and service targets.
EDUCATION
University of Washington 09 2022 – 03 2025
Bachelor of Science in Computer Science -CGPA - 3.9 Tacoma, Washington
Bellevue College 09 2019 – 06 2021
Running Start Bellevue, Washington
EXTRACURRICULAR
• Participated in UWT SEP 2023 Programming Contest
• Participated in UWT MAY 2024 Programming Contest


